### Homework 2
Modern Stellar Astrophysics 

Prof. Melissa Ness

Yasmeen Asali 

October 18th, 2018

In [45]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

### Problem 2



In [19]:
dt = [('x','f8'),('y','f8'),('z','f8'),('vx','f8'),('vy','f8'),('vz','f8')]
data = np.genfromtxt('modelR1GaiaChallenge',dtype=dt)
np.save('modelR1data',data)

(7489269,)


In [34]:
modelR1data = np.load('modelR1data.npy')
print(np.shape(modelR1data))

(7489269,)


In [35]:
modelR1data

array([ (-1.1855888 ,  0.3870438 ,  0.28164312,  -97.19944,   15.530616, -151.94402 ),
       ( 0.40000296,  0.24807201, -0.05270057, -175.68466,  223.81343 ,  184.67241 ),
       (-1.0759624 , -0.55642074, -0.07496975,  135.59796, -137.67535 ,  112.96014 ),
       ...,
       ( 5.8122311 ,  5.6434278 , -0.08249884, -156.72023,   19.514315,  -34.014992),
       (-5.9588952 , -5.5228796 , -0.02548528,  157.02412,  -21.07032 ,  -27.067957),
       ( 5.6456079 ,  5.5886564 ,  0.08627713, -162.69707,   14.673622,  -19.854509)],
      dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8')])

In [32]:
# how to make maps to see the central X-shaped structure
#we have coords in xyz per star, along with their velocities
# probably just plotting the starts in the x-z plane to start will be ok? 

In [48]:
x = modelR1data['x']
y = modelR1data['y']
z = modelR1data['z']

In [ ]:
#use a numpy function to np.random.rand to create an array that goes 
#from 0 to the total number of stars
#with some 10,000 stars 

#subset of the data 

np.random.randit(0, 10, 100) # so 0 to 7 million
#100 is the number of points you want ex 10,000

#index your x,y,z
#before you do this, take x,y,z less that five (kpc)
#don't do it for 7 million 
#2d histogram 
#unsharp mask - subtract instead of divide if in log 
#divide if density is in linear scale 

In [ ]:
#point density in xz plane
xz_num = np.vstack([x,z])
c_num = gaussian_kde(xz_num)(xz_num)

In [ ]:
#sort by density, so that the densest points are plotted last
idx = c_num.argsort()
x, y, c_num = x[idx], y[idx], c_num[idx]

In [46]:
plt.plot(x, z, c=c_num, cmap='Oranges')
plt.xlabel("x-axis",size=15)
plt.ylabel("z-axis",size=15)
cb = plt.colorbar()
cb.set_label('Number Density',fontsize=15)

NameError: name 'c_num' is not defined

### Problem 3

**Part a)**

The Kelvin Helmholtz mechanism theorizes that stars or planets (Jupiter in this case) that are not in hydrostatic equilibrium will undergo contraction as the surface cools, causing an increase in core temperature which prevents the planet from collapsing immediately. The thermal timescale is the amount of time this mechanism can persist before all of Jupiter's energy is radiated away. It can be calculated using the following formula:  

$$ \tau_{th} = \frac{\Delta{E_{g}}}{L} = \frac{GM^2}{RL} $$

Solving this equation below, we find Jupiter's thermal timescale is 321 gigayears. Since Jupiter is only 4.5 Gyr old, gravitational contraction could have power its luminosity for its entire lifetime, so we cannot assume it is in hydrostatic equilibrium. 

In [3]:
def ts_thermal(M, R, L):
    G = 6.64e-8 * (u.cm)**3 / (u.g * (u.s)**(2))
    t_in_seconds = (G*(M**2))/(R*L)
    t_in_Gyr = t_in_seconds * (3.17e-17 * u.Gyr / u.s)
    return t_in_Gyr

M_jup = 1.9e30 * u.g
R_jup = 7.1e9 * u.cm
L_jup = 8.7e-10 * (3.83e33 * u.g * (u.cm)**2 * (u.s)**(-3))

print(ts_thermal(M_jup, R_jup, L_jup))

321.18715474021167 Gyr


**Part b)**

If Jupiter's radius is shrinking only due to radiation we can set up conservation of energy by equating the amount of energy lost in gravitational potential energy to the amoutn of energy emitted in luminosity:

$$ \Delta{E_{g}} = \Delta{E_{emitted}} $$
$$ \frac{GM^2}{r(t)} = Lt $$

We can take the time derivative of both sides, noting that since the star is contracting, radius is no longer a constant but now a function of time. 

$$ \frac{d}{dt}(\frac{GM^2}{r(t)}) = L $$

Using the recipricol rule for derivatives

$$ \frac{-GM^2}{r(t)^2}\frac{dr(t)}{dt} = L $$
$$ \frac{dr(t)}{dt} = \frac{-Lr(t)^2}{GM^2} $$

We are trying to find the rate at a given time (now), so we can substitute $ r(t) = R_{jupiter} $. Also, since we know that the radius is contracting, we can ignore the negative sign and calculate the rate

In [30]:
def contr_rate(M, R, L):
    G = 6.64e-8 * (u.cm)**3 / (u.g * (u.s)**(2))
    return (L*R**2)/(G*M**2)
def convert_to_year_per_sec(value):
    return value * (u.s / (3.17098e-8 * u.yr))

print(contr_rate(M_jup, R_jup, L_jup))
print(convert_to_year_per_sec(contr_rate(M_jup, R_jup, L_jup)))

7.007440885425358e-10 cm / s
0.022098659989736164 cm / yr


Our value is 0.02 cm per year, which is 2 orders of magnitude off of the actual value of 2 cm per year (**source**). This offset can be attributed to the fact that we assumed a simplified equation of the gravitational potential energy, ignoring the factor that arises from the density distribution. 

### Problem 4



### Problem 5

**Part a)**

The mean free path ${l}$ tells us how far we can see through the Earth's atmosphere, and it is given by:

$$ l = (\kappa\rho)^{-1} $$

Plugging in the given values we get:

In [9]:
op_E = 0.264 * u.cm**2 / u.g
dens_E = 1.2e-3 * u.g / u.cm**3
mfp_E = 1 / (op_E*dens_E)

print(mfp_E)

3156.5656565656564 cm


** Part b) **

Using the same formula, we can calculate the mean free path for a photon in the sun

In [23]:
op_S = 1.16 * u.cm**2 / u.g
dens_S = 162 * u.g / u.cm**3
mfp_S = 1 / (op_S*dens_S)

print(mfp_S)

0.005321413367390379 cm


** Part c) **

Distance travelled by a photon can be expresed as:

$$ d = \sqrt{N}l $$

For a photon coming from the center of the Sun to its surface, $ d = R_{\odot} $. So the number of steps is

$$ N = \frac{R_{\odot}^2}{l^2} $$

So here $N$ signifies the number of random steps a photon. This photon will travel a distance $l$ per random step, so the total distance travelled is

$$ D = Nl $$

and so we can express the amount of time it takes (considering the constant speed of light) as:

$$ t = \frac{Nl}{c} = \frac{R_{\odot}^2}{lc} $$

Plugging in our values we get that it will take a photon 960,393 years to travel from the center to the surface of the Sun. Crazy! 

In [31]:
R_sun = 6.95e10 * u.cm
c = 2.997e10 * u.cm / u.s
t_photon = (R_sun**2 ) / (mfp_S * c)

def convert_to_year(value):
    return value * (3.17098e-8 * u.yr) / u.s

print(t_photon)
print(convert_to_year(t_photon))

30286972972972.973 s
960393.8555783783 yr
